In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkSession available as 'spark'.


In [2]:
from graphframes import * 

vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sqlCtx.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

gf = GraphFrame(vertices, edges)


In [4]:
gf.vertices.filter("age > 30").show()

+---+----+---+------+----------+
| id|name|age|gender|university|
+---+----+---+------+----------+
|  5|Oleg| 35|     M|      MIPT|
+---+----+---+------+----------+



In [4]:
gf.inDegrees.filter("inDegree > 2").show()

+---+--------+
| id|inDegree|
+---+--------+
|  3|       6|
|  1|       3|
|  2|       3|
+---+--------+



In [5]:
gf.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  7|       1|
|  3|       6|
|  5|       2|
|  6|       1|
|  1|       3|
|  4|       2|
|  2|       3|
+---+--------+



In [6]:
gf.find("(a)-[e]->(b); (b)-[e2]->(a)").show()

+--------------------+------------+--------------------+------------+
|                   a|           e|                   b|          e2|
+--------------------+------------+--------------------+------------+
|  [1,Alex,28,M,MIPT]|[1,4,friend]| [4,Pavel,30,M,MIPT]|[4,1,friend]|
| [4,Pavel,30,M,MIPT]|[4,1,friend]|  [1,Alex,28,M,MIPT]|[1,4,friend]|
|[3,Natasha,27,F,S...|[3,2,friend]| [2,Emeli,28,F,MIPT]|[2,3,friend]|
| [2,Emeli,28,F,MIPT]|[2,1,friend]|  [1,Alex,28,M,MIPT]|[1,2,friend]|
| [2,Emeli,28,F,MIPT]|[2,5,friend]|  [5,Oleg,35,M,MIPT]|[5,2,friend]|
|[3,Natasha,27,F,S...|[3,5,friend]|  [5,Oleg,35,M,MIPT]|[5,3,friend]|
|  [1,Alex,28,M,MIPT]|[1,3,friend]|[3,Natasha,27,F,S...|[3,1,friend]|
|[3,Natasha,27,F,S...|[3,1,friend]|  [1,Alex,28,M,MIPT]|[1,3,friend]|
|  [5,Oleg,35,M,MIPT]|[5,3,friend]|[3,Natasha,27,F,S...|[3,5,friend]|
| [2,Emeli,28,F,MIPT]|[2,3,friend]|[3,Natasha,27,F,S...|[3,2,friend]|
|[3,Natasha,27,F,S...|[3,7,friend]|   [7,Ilya,29,M,MSU]|[7,3,friend]|
| [4,Pavel,30,M,MIPT

In [7]:
motifs = gf.find("(a)-[]->(b); (b)-[]->(c)")
motifs.show()

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
| [2,Emeli,28,F,MIPT]|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [2,Emeli,28,F,MIPT]|  [

In [7]:
motifs = gf.find("(A)-[]->(B); (B)-[]->(C)").filter("A.id != C.id")
motifs.show()

+--------------------+--------------------+--------------------+
|                   A|                   B|                   C|
+--------------------+--------------------+--------------------+
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [2,Emeli,28,F,MIPT]|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
| [4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [2,Emeli,28,F,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
|  [1,Alex,28,M,MIPT]| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|
|  [5,Oleg,35,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
|  [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|
|   [6,Ivan,30,M,MSU]|[3,

In [16]:
AC = motifs.selectExpr("A.id as A", "C.id as C")
AC.show()

+---+---+
|  A|  C|
+---+---+
|  3|  1|
|  5|  1|
|  2|  1|
|  4|  1|
|  5|  1|
|  6|  1|
|  7|  1|
|  3|  1|
|  3|  2|
|  4|  2|
|  1|  2|
|  4|  2|
|  5|  2|
|  6|  2|
|  7|  2|
|  3|  2|
|  2|  3|
|  4|  3|
|  1|  3|
|  5|  3|
+---+---+
only showing top 20 rows



In [17]:
AC.groupBy("A", "C").count().filter("A = 1").show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  2|    1|
|  1|  3|    2|
|  1|  4|    1|
|  1|  5|    2|
|  1|  6|    1|
|  1|  7|    1|
+---+---+-----+



In [ ]:
AC.groupBy("A", "C").count().show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  4|  1|    1|
|  4|  2|    2|
|  4|  3|    1|
|  4|  5|    1|
|  4|  6|    1|
|  4|  7|    1|
|  5|  1|    2|
|  5|  2|    1|
|  5|  3|    1|
|  5|  4|    1|
|  5|  6|    1|
|  5|  7|    1|
|  6|  1|    1|
|  6|  2|    1|
|  6|  4|    1|
|  6|  5|    1|
|  6|  7|    1|
|  1|  2|    1|
|  1|  3|    2|
|  1|  4|    1|
+---+---+-----+
only showing top 20 rows



In [8]:
results = gf.triangleCount()
results.show()

+-----+---+-------+---+------+----------+
|count| id|   name|age|gender|university|
+-----+---+-------+---+------+----------+
|    0|  7|   Ilya| 29|     M|       MSU|
|    3|  3|Natasha| 27|     F|     SPbSU|
|    1|  5|   Oleg| 35|     M|      MIPT|
|    0|  6|   Ivan| 30|     M|       MSU|
|    2|  1|   Alex| 28|     M|      MIPT|
|    1|  4|  Pavel| 30|     M|      MIPT|
|    2|  2|  Emeli| 28|     F|      MIPT|
+-----+---+-------+---+------+----------+



### Page Rank

In [14]:
results = gf.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.show()

+---+-------+---+------+----------+------------------+
| id|   name|age|gender|university|          pagerank|
+---+-------+---+------+----------+------------------+
|  1|   Alex| 28|     M|      MIPT| 0.925034321035054|
|  3|Natasha| 27|     F|     SPbSU|1.8587232404145666|
|  2|  Emeli| 28|     F|      MIPT| 0.925034321035054|
|  4|  Pavel| 30|     M|      MIPT|0.6561021410596017|
|  7|   Ilya| 29|     M|       MSU|0.4038005647844105|
|  6|   Ivan| 30|     M|       MSU|0.4038005647844105|
|  5|   Oleg| 35|     M|      MIPT|0.6561021410596017|
+---+-------+---+------+----------+------------------+



In [16]:
results = gf.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.show()

+---+-------+---+------+----------+-------------------+
| id|   name|age|gender|university|           pagerank|
+---+-------+---+------+----------+-------------------+
|  1|   Alex| 28|     M|      MIPT|   1.01935961862719|
|  3|Natasha| 27|     F|     SPbSU| 2.0084281384336697|
|  2|  Emeli| 28|     F|      MIPT|   1.01935961862719|
|  4|  Pavel| 30|     M|      MIPT| 0.7202143452359088|
|  7|   Ilya| 29|     M|       MSU|0.43337602947710424|
|  6|   Ivan| 30|     M|       MSU|0.43337602947710424|
|  5|   Oleg| 35|     M|      MIPT| 0.7202143452359088|
+---+-------+---+------+----------+-------------------+



### Connected components

In [15]:
vertices = spark.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = spark.createDataFrame([
  ("1", "2", "friend"),
  ("1", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

reduced_g = GraphFrame(vertices, edges)

In [13]:
spark.sparkContext.setCheckpointDir("/tmp")
result = reduced_g.connectedComponents()
result.select("id", "component").orderBy("component").show()

+---+-------------+
| id|    component|
+---+-------------+
|  3|  25769803776|
|  7|  25769803776|
|  5|  25769803776|
|  6|  25769803776|
|  2|1236950581248|
|  4|1236950581248|
|  1|1236950581248|
+---+-------------+



###  People you may know

In [23]:
from pyspark.sql.functions import abs, col, explode, split
from pyspark.sql.types import *

In [17]:
dataPath = "/Users/pritykovskaya/Desktop/NewProLab/"
graphPath = dataPath + "trainGraph/"
usersToPredictPath = dataPath + "prediction.csv"

In [21]:
data = spark.read.format("csv").option("delimiter", "\t")\
    .load(graphPath).withColumnRenamed("_c0", "user").withColumnRenamed("_c1", "friendsString")
    

In [24]:
from pyspark.sql.functions import udf

def cutStartEndBrackets(s):
    return s[2:-2]

cutStartEndBracketsUDF = udf(cutStartEndBrackets, StringType())

userFriend = \
    data.select(col("user"), split(cutStartEndBracketsUDF(col("friendsString")), "\),\(").alias("friendsMasks"))\
    .withColumn("friendMask", explode('friendsMasks'))\
    .withColumn("dst", split(col("friendMask"), ",")[0])\
    .withColumn("src", col("user"))\
    .select(col("src").cast("integer"), col("dst").cast("integer"))

userFriendSymmetric = \
    userFriend\
        .withColumn("tmp", col("src"))\
        .withColumn("src", col("dst"))\
        .withColumn("dst", col("tmp"))\
        .select("src", "dst")
        
edges = userFriend.unionAll(userFriendSymmetric)

vertices = edges.select(col("src").alias("id")).distinct()

In [25]:
usersFriends = GraphFrame(vertices, edges)